In [1]:
!pip install biopython

In [2]:
### Import **Libraries**

import os
import os.path
import numpy as np
import re
from numpy import array
import subprocess
import math
import time
import pdb
import shutil
import networkx as nx
import matplotlib.pyplot as plt
import sys
from collections import Counter
from Bio.PDB import MMCIFParser, PDBParser, Selection, NeighborSearch, Entity

In [3]:
def pdb_model(structure_file, water=False):
    """Return a biopython [1] model entity from a structure file.

    Parameters
    ----------
    structure_file: string
        Path to structure file
    water: boolean (default=False)
        True to take into account waker molecules in the structure, False
        otherwise.

    Notes
    -----
    1. http://biopython.org/wiki/Biopython

    """
    accepted_formats = ['cif', 'pdb', 'ent']
    parsers = [MMCIFParser, PDBParser, PDBParser]
    protein_name, file_format = structure_file.rsplit('.', 1)

    try:
        parser = parsers[accepted_formats.index(file_format)]
        parser = parser(QUIET=True)
    except ValueError:
        raise Exception("Accepted structure files are: {}".format(
            accepted_formats))

    structure = parser.get_structure(protein_name, structure_file)
    model = structure[0]

    if not water:
        for chain in model.get_chains():
            for residue in list(chain):
                hetero_flag = residue.id[0].strip()
                # Empty strings evaluate to False.  Therefore hetero_flag
                # returns False if the residue is not a water molecule.
                if hetero_flag:
                    chain.detach_child(residue.id)
            if not list(chain):
                model.detach_child(chain.id)

    return model


def label_residue(residue):
    """ Return a string of the label of the biopython [1] residue object.

    The label of the residue is the following:
        Chain + Position

    Parameters
    ----------
    residue: Bio.PDB.Residue.Residue
        The residue to be labeled.

    Notes
    -----
    1. http://biopython.org/wiki/Biopython

    """
    position = str(residue.id[1])
    chain = residue.parent.id

    return chain + ' ' + position #added space deliminater for split function.

## Residue adjecmatrixtest

In [4]:
#Biopython used to create adjecency dictionary
def residue_adjacency_CO_test(model, cutoff=5, weight=True):
    """Return residue adjacency dictionary defined by cutoff distance.

    Parameters
    ----------
    model: Bio.PDB.Model
        Model created with the atomic coordinates of the protein file.

    cutoff: int or float
        Distance cutoff defining links between atoms.  Two atoms are adjacent
        if their distance is less than the given cutoff.

    See Also
    --------`````````````
    pdb_model

    """
    #Use only the Atoms specified in paper
    atom_list = Selection.unfold_entities(model, 'A')
    atoms = [atom for atom in atom_list if 'O' in atom.name or 'C' in atom.name] # or 'C' in atom.name
    #Only looking at oxygen for cliques at the momement, for maximum clique finidng in the residue filteration setup
#     print(atoms)

    neighbor_search = NeighborSearch(atoms)
    atomic_adjacency = {}

    for atom in atoms:
        _res = label_residue(atom.get_parent())
        adjacent_atoms = []
        for adj_atom in neighbor_search.search(atom.coord, cutoff):
            _adj_res = label_residue(adj_atom.parent)
            # Adjacent atoms must be in different residues
            if _adj_res != _res:
                adjacent_atoms.append(adj_atom)
        atomic_adjacency[atom] = adjacent_atoms

    adjacency = {}

    # Create residue adjacency dictionary with string format, see
    # label_residue.
    for atom, neighbors in atomic_adjacency.items():
        residue = label_residue(atom.get_parent())
        adjacency.setdefault(residue, [])

        # Only different residues are connected by an edge (No loops).
        not_in_residue = []
        for neighbor in neighbors:
            neighbor_parent = label_residue(neighbor.get_parent())
            if neighbor_parent is not residue:
                not_in_residue.append(neighbor_parent)

        adjacency[residue].extend(not_in_residue)

    if not weight:

        return adjacency

    # Make new dictionary mapping each residue to its neighbors taking
    # into account the weight.
    weighted_adjacency = {}
    for residue in adjacency:
        counter = Counter(adjacency[residue])
        weighted_adjacency[residue] = {
            neighbor: {'weight': counter[neighbor]}
            for neighbor in counter}

    return weighted_adjacency

In [5]:
def topo_network(model, cutoff=4, weight=True):
    """Return the interaction network of a protein structure.

    The interaction network is defined by a distance cutoff.

    Parameters
    ----------
    model: Bio.PDB.model
        The protein structure.
    cutoff: float
        The distance cutoff defining an interaction between two nodes.
    weight: boolean
        True if atomic interactions are to be considered.
    """

    adjacency_dictionary = residue_adjacency_CO_test(model, cutoff=cutoff,
                                            weight=weight)

    return nx.Graph(adjacency_dictionary)


In [6]:
class Pmolecule(object):
    """Create a Pmolecule object.

    The Pmolecule calls a number of methods for the analysis of protein
    structure. This includes the construction of the interaction network of the
    protein.

    Parameters
    ----------
    structure_file = str
        The path to the structure file of the targeted protein. Three
        structure-file formats are accepted: `pdb', `cif', and `ent'.
    water: boolean, default is False
        If false, water molecules are ignored.

    Attributes
    ----------
    model: Bio.PDB.model
        The structural model of the structure. See www.biopython.org.
    path_to_file: str
        The path to the structural file used to instantiate the class.
    """

    def __init__(self, structure_file, water=False):
        self.model = pdb_model(structure_file, water=water)
        self.path_to_file = structure_file

    def get_network(self, cutoff=5, weight=True):
        """Return the interaction network of a protein structure.

        The interaction network is defined by a distance cutoff.

        Parameters
        ----------
        model: Bio.PDB.model
            The protein structure.
        cutoff: float
            The distance cutoff defining an interaction between two nodes.
        weight: boolean
            True if atomic interactions are to be considered.
        """

        return topo_network(self.model, cutoff=cutoff, weight=weight)

    def get_pdbmodel(self):
        return self.model

In [7]:
########################################################################################################

In [8]:
class FileOperation(object):
    """ generated source for class FileOperation """

    @classmethod
    def getEntriesAsList(cls, fileName):
        """ generated source for method getEntriesAsList """
        #  Return list entries. 
        #  An element of that list is one line of the file fileName.
        fil = open(fileName, "r")
        entries = fil.read().split('\n')
        fil.close()
        return entries

    @classmethod
    def saveResults(cls, entryNames, fileName, option):
        """ generated source for method saveResults """
        #  Write list entryNames in file fileName.
        #  An element in entryNames is one line in fileName.
        if option == "w":
            try:
                with open(fileName, "w") as f:
                    i = 0
                    while i < len(entryNames):
                        f.write(str(entryNames[i]) + "\n")
                        i += 1
            except Exception as e:
                pass

In [9]:
class ParsePDB(object):
    """ generated source for class ParsePDB """
    @classmethod
    def getLines(cls, PDBFile, field, subfield, atom):
        """ generated source for method getLines """
        #  Return a list, the element of which is a line in the file with name PDBFile
        #  that contains the specified atom in a specified field.
        returnList = []
        fileList = []
        line = ""
        i = 0
        if atom != "allatom":
            if (0 == len(atom)) or (len(atom) >= 4):
                print("error! in getLines")
        fileList = FileOperation.getEntriesAsList(PDBFile)
        while i < len(fileList):
            line = str(fileList[i])
            if field == "HETATM" and subfield != "HOH" and line.startswith(field) and line[17:20].strip() in atom:
                returnList.append(line)
            if field == "ATOM" and subfield == "Res" and line.startswith(field) and line[13:17].startswith(atom):
                returnList.append(line)
            if field == "HETATM" and subfield == "HOH" and line.startswith(field) and line[12: 14].startswith(atom) and line[17: 20].startswith(subfield):
                returnList.append(line)
            if subfield == "HET":
                if field == "HETATM" and subfield != "HOH" and line.startswith(field) and line[12:14].startswith(atom) and ("HOH" in line[17: 20]) == False:
                    returnList.append(line)
            if field == "ATOM" and subfield == "ResNoWater" and (line.startswith("ATOM") or line.startswith("HETATM")) and atom == "allatom":
                if ("HOH" in line[17:20]) == False:
                    returnList.append(line)
            if field == "ATOM" and subfield == "Res" and (line.startswith("ATOM") or line.startswith("HETATM")) and atom == "allatom":
                returnList.append(line)
            i += 1
        return returnList

    @classmethod
    def getBondedLines(cls, PDBFile, atom, bondedAtom):
        """ generated source for method getBondedLines """
        returnList = []
        carbonList = []
        oxygenList = []
        fileList = []
        line = ""
        i = 0
        if 0 == len(atom) or len(atom) > 1 or 0 == len(bondedAtom) or len(bondedAtom) > 1:
            print("error! in getBondedLines")
        atom = atom.upper()
        bondedAtom = bondedAtom.upper()
        temp = ""
        fileList = FileOperation.getEntriesAsList(PDBFile)
        while i < len(fileList):
            line = str(fileList[i])
            if ("ATOM" in line[0:4]) and (bondedAtom in line[13]):
                carbonList.append(line)
                if temp == "" and (len(oxygenList)==0) == False:
                    k = 0
                    while k < len(oxygenList):
                        returnList.append(str(oxygenList[k]))
                        k += 1
                # print(oxygenList)
                # oxygenList = []
                temp = line[13:14]
            if ("ATOM" in line[0:4]) and (atom in line[13]):
                oxygenList.append(line)
                if temp == "C" and ((len(carbonList)==0) == False):
                    k = 0
                    while k < len(carbonList):
                        returnList.append(str(carbonList[k]))
                        k += 1
                # print(carbonList)
                # carbonList = []
                temp = line[13:14]
            if (("TER" in line[0:4]) or ("HETATM" in line[0:6]) and (0 != len(temp))):
                k = 0
                while k < len(oxygenList):
                    returnList.append(str(oxygenList[k]))
                    k += 1
                # print(oxygenList)
                # oxygenList = []
                temp = ""
            i += 1
        return returnList

In [10]:
def runPrediction(filenamearg, metal, PDBID, resultAtoms):
    """ generated source for method getMetal """
    fileList = []
    resultList = []
    allAtom = []
    allAtomNoWater = []
    currDirectory = os.getcwd()
    fileList = FileOperation.getEntriesAsList(filenamearg)
    #  a list of PDBIDs   
    # print(args);
    fileName = currDirectory + "/inputdata/" + PDBID + ".pdb"
    caList = ParsePDB.getLines(fileName, "HETATM", metal, metal)
    # print("This is the caList", caList)
    allAtom = ParsePDB.getLines(fileName, "ATOM", "Res", "allatom")
    pdb_site_list = allAtom + resultAtoms
    FileOperation.saveResults(pdb_site_list, currDirectory + "/predictionResults/" + PDBID + "_site.pdb", "w")
    print("File: " + currDirectory + "/predictionResults/" + PDBID + "_site.pdb" + " just written")
    print('Completed!')
    resultList = []


In [11]:
def center_of_mass(entity, geometric=False):
    """
    Returns gravitic [default] or geometric center of mass of an Entity.
    Geometric assumes all masses are equal (geometric=True)
    """
    
    # Structure, Model, Chain, Residue
    if isinstance(entity, Entity.Entity):
        atom_list = entity.get_atoms()
    # List of Atoms
    elif hasattr(entity, '__iter__') and [x for x in entity if x.level == 'A']:
        atom_list = entity
    else: # Some other weirdo object
        raise ValueError("Center of Mass can only be calculated from the following objects:\n"
                            "Structure, Model, Chain, Residue, list of Atoms.")
    
    masses = []
    positions = [ [], [], [] ] # [ [X1, X2, ..] , [Y1, Y2, ...] , [Z1, Z2, ...] ]
    
    for atom in atom_list:
        masses.append(atom.mass)
        
        for i, coord in enumerate(atom.coord.tolist()):
            positions[i].append(coord)

    # If there is a single atom with undefined mass complain loudly.
    if 'ukn' in set(masses) and not geometric:
        raise ValueError("Some Atoms don't have an element assigned.\n"
                         "Try adding them manually or calculate the geometrical center of mass instead.")
    
    if geometric:
        return [sum(coord_list)/len(masses) for coord_list in positions]
    else:       
        w_pos = [ [], [], [] ]
        for atom_index, atom_mass in enumerate(masses):
            w_pos[0].append(positions[0][atom_index]*atom_mass)
            w_pos[1].append(positions[1][atom_index]*atom_mass)
            w_pos[2].append(positions[2][atom_index]*atom_mass)

        return [sum(coord_list)/sum(masses) for coord_list in w_pos]

In [12]:
# def get_Metal():
#     meList = ParsePDB.getLines(fileName, "HETATM", metal, metal)

## Experimental

In [13]:
PDBID = '3cln'
metal = 'CA'

In [14]:
def setup(arg):
    pdb_id = arg
    commands = 'mkdir -p ./inputdata; mkdir -p ./predictionResults; cd inputdata; wget https://files.rcsb.org/download/'+ pdb_id + '.pdb; echo "'+ pdb_id + '" >> list.txt; cat list.txt; cd .. echo "Created all needed folders";'
    p = subprocess.Popen(commands, shell=True, stdout=subprocess.PIPE)
    p.wait()
  # print(str(p.communicate()[0], 'utf-8'))

setup(PDBID)

In [15]:
fileName = './inputdata/'+ PDBID + '.pdb'
caListtest = ParsePDB.getLines(fileName, "HETATM", metal, metal)
caListtest

['HETATM 1128 CA    CA A 149     -13.638  53.352  31.500  1.00 12.29          CA  ',
 'HETATM 1129 CA    CA A 150     -17.797  50.156  20.859  1.00 15.64          CA  ',
 'HETATM 1130 CA    CA A 151      15.035  17.711  17.954  1.00 28.17          CA  ',
 'HETATM 1131 CA    CA A 152      13.784  23.887   8.345  1.00 29.20          CA  ']

In [16]:
def vecCaList(caListtest):
    fList = []
    for caitem in caListtest:
        txt=caitem
        re1='.*?'	# Non-greedy match on filler
        re2='([+-]?\\d*\\.\\d+)(?![-+0-9\\.])'	# Float 1
        re3='.*?'	# Non-greedy match on filler
        re4='([+-]?\\d*\\.\\d+)(?![-+0-9\\.])'	# Float 2
        re5='.*?'	# Non-greedy match on filler
        re6='([+-]?\\d*\\.\\d+)(?![-+0-9\\.])'	# Float 3
        rg = re.compile(re1+re2+re3+re4+re5+re6,re.IGNORECASE|re.DOTALL)
        m = rg.search(txt)
        if m:
            float1=m.group(1)
            float2=m.group(2)
            float3=m.group(3)
            print ("("+float1+")"+"("+float2+")"+"("+float3+")"+"\n")
        plhld = [float(float1), float(float2), float(float3)]
        ac = array(plhld)
        fList.append(ac)
    return fList

In [17]:
caList = vecCaList(caListtest)

(-13.638)(53.352)(31.500)

(-17.797)(50.156)(20.859)

(15.035)(17.711)(17.954)

(13.784)(23.887)(8.345)



In [18]:
caList

[array([-13.638,  53.352,  31.5  ]),
 array([-17.797,  50.156,  20.859]),
 array([15.035, 17.711, 17.954]),
 array([13.784, 23.887,  8.345])]

In [19]:
molecule = Pmolecule('./inputdata/'+ PDBID + '.pdb')

In [20]:
# Create Biograph, uses Biopython for stucture and networkx for drawing the graphs
Graph = molecule.get_network()
# nx.draw(Graph, with_labels = True) #, with_labels = True

In [21]:
# nx.draw_circular(Graph)

In [22]:
model = molecule.get_pdbmodel()

In [23]:
adjmatrix = residue_adjacency_CO_test(model, cutoff=4, weight=True)

In [24]:
# adjmatrix

In [25]:
def get_res_cliqs(G, M):
    # Find max cliques
    renamed_cliques=[]
    cliques=[clique for clique in nx.find_cliques(G) if len(clique)>= 4]
    for cliq in cliques:
        c=[]
        for cli in cliq:
            res = M[str(cli.split(' ')[0])][int(cli.split(' ')[1])]
            if (res.get_resname() == 'ASP') or (res.get_resname() == 'GLU') or (res.get_resname() == 'ASN') or (res.get_resname() == 'GLN') or (res.get_resname() == 'SER') or (res.get_resname() == 'THR'):
                c.append(res)
            else:
                continue
        renamed_cliques.append(c)
    print('Number of cliques found: ',len(cliques))
    return(renamed_cliques)

In [26]:
reslist = get_res_cliqs(Graph, model)

Number of cliques found:  208


In [27]:
reslist

[[<Residue GLU het=  resseq=104 icode= >],
 [<Residue GLU het=  resseq=104 icode= >],
 [<Residue GLU het=  resseq=104 icode= >,
  <Residue SER het=  resseq=101 icode= >],
 [<Residue ASP het=  resseq=64 icode= >,
  <Residue GLU het=  resseq=67 icode= >],
 [<Residue ASP het=  resseq=64 icode= >],
 [],
 [<Residue GLU het=  resseq=67 icode= >],
 [],
 [<Residue THR het=  resseq=70 icode= >],
 [<Residue THR het=  resseq=70 icode= >,
  <Residue GLU het=  resseq=67 icode= >],
 [<Residue THR het=  resseq=70 icode= >,
  <Residue GLU het=  resseq=67 icode= >],
 [],
 [],
 [],
 [<Residue ASP het=  resseq=64 icode= >,
  <Residue GLU het=  resseq=67 icode= >],
 [<Residue ASP het=  resseq=64 icode= >],
 [],
 [],
 [],
 [],
 [<Residue THR het=  resseq=34 icode= >],
 [],
 [],
 [<Residue THR het=  resseq=34 icode= >],
 [<Residue GLN het=  resseq=41 icode= >],
 [<Residue SER het=  resseq=38 icode= >,
  <Residue THR het=  resseq=34 icode= >],
 [<Residue SER het=  resseq=38 icode= >],
 [<Residue SER het=  re

## Maximal Cliques, Carbon Clusters

In [28]:
def centreCa2(resatoms):
    ans = []
    for i, (atomlist) in enumerate(resatoms):
        if not atomlist:
            continue
        elif len(atomlist) > 2:
            ans.append(center_of_mass(atomlist))
    return ans

In [29]:
#Given a group of residues, for each atom in the residue return the one with the min(dist) from the any of the HETAM CA
def positionDistFilter(reslist, caList, model):
    # First get oxygen in each residue
    limit = 1.74
    cutoff = 2.5
    resoxygrouped = []
    for resli in reslist:
        grup = []
        for res in resli:
            atom_list = res.get_list()
            resatoms = [atom for atom in atom_list if 'O' in atom.name]
            print(resatoms)
            for acvec in caList:
                alpha_carbon = acvec
                distances = []
                resoxygroup = []
                for i,(atom) in enumerate(resatoms):
                    #subtract the two position vectors
                    print(atom.get_coord())
                    diff_vector = alpha_carbon - atom.get_coord()
                    #to get a positive value we square the difference vector
                    #we then take the square root to go back to the original scale
                    distances.append(np.sqrt(np.sum(diff_vector * diff_vector)))
                    dist = np.sqrt(np.sum(diff_vector * diff_vector))
                    print(i)
                    print('The distances are: ',distances)
                    #we get the nearest atom using min(distances) or dist and see if it falls inside
                    #the cutoff
                    if (0 < dist) and (dist > limit) and (dist < cutoff):
                        grup.append(atom)
                    print(res)
        resoxygrouped.append(grup)
    return resoxygrouped
#finding Ca binding site from remaining max clique residue


In [30]:
positionDistFilterans = positionDistFilter(reslist, caList, model)

[<Atom O>, <Atom OE1>, <Atom OE2>]
[10.69  10.994 15.506]
0
The distances are:  [51.39902470527058]
<Residue GLU het=  resseq=104 icode= >
[14.063 15.928 16.839]
1
The distances are:  [51.39902470527058, 48.81440390407834]
<Residue GLU het=  resseq=104 icode= >
[14.887 15.027 18.597]
2
The distances are:  [51.39902470527058, 48.81440390407834, 49.48705515474998]
<Residue GLU het=  resseq=104 icode= >
[10.69  10.994 15.506]
0
The distances are:  [48.7219249279762]
<Residue GLU het=  resseq=104 icode= >
[14.063 15.928 16.839]
1
The distances are:  [48.7219249279762, 46.93373978324205]
<Residue GLU het=  resseq=104 icode= >
[14.887 15.027 18.597]
2
The distances are:  [48.7219249279762, 46.93373978324205, 48.035477679692114]
<Residue GLU het=  resseq=104 icode= >
[10.69  10.994 15.506]
0
The distances are:  [8.365991748352537]
<Residue GLU het=  resseq=104 icode= >
[14.063 15.928 16.839]
1
The distances are:  [8.365991748352537, 2.3166992388980705]
<Residue GLU het=  resseq=104 icode= >
[

0
The distances are:  [10.357958081912141]
<Residue ASP het=  resseq=64 icode= >
[-21.     47.883  23.974]
1
The distances are:  [10.357958081912141, 11.863796750283292]
<Residue ASP het=  resseq=64 icode= >
[-19.3    46.488  24.165]
2
The distances are:  [10.357958081912141, 11.863796750283292, 11.531476799344938]
<Residue ASP het=  resseq=64 icode= >
[-17.2    45.584  25.647]
0
The distances are:  [6.647144747989269]
<Residue ASP het=  resseq=64 icode= >
[-21.     47.883  23.974]
1
The distances are:  [6.647144747989269, 5.012880552608857]
<Residue ASP het=  resseq=64 icode= >
[-19.3    46.488  24.165]
2
The distances are:  [6.647144747989269, 5.012880552608857, 5.161674183042715]
<Residue ASP het=  resseq=64 icode= >
[-17.2    45.584  25.647]
0
The distances are:  [43.30336732762823]
<Residue ASP het=  resseq=64 icode= >
[-21.     47.883  23.974]
1
The distances are:  [43.30336732762823, 47.38260424688753]
<Residue ASP het=  resseq=64 icode= >
[-19.3    46.488  24.165]
2
The distanc

0
The distances are:  [37.424390892463926]
<Residue GLU het=  resseq=67 icode= >
[-17.202  47.995  21.937]
1
The distances are:  [37.424390892463926, 41.546001722939316]
<Residue GLU het=  resseq=67 icode= >
[-18.502  48.046  20.243]
2
The distances are:  [37.424390892463926, 41.546001722939316, 42.04290193659768]
<Residue GLU het=  resseq=67 icode= >
[<Atom O>, <Atom OD1>, <Atom OD2>]
[-17.2    45.584  25.647]
0
The distances are:  [10.357958081912141]
<Residue ASP het=  resseq=64 icode= >
[-21.     47.883  23.974]
1
The distances are:  [10.357958081912141, 11.863796750283292]
<Residue ASP het=  resseq=64 icode= >
[-19.3    46.488  24.165]
2
The distances are:  [10.357958081912141, 11.863796750283292, 11.531476799344938]
<Residue ASP het=  resseq=64 icode= >
[-17.2    45.584  25.647]
0
The distances are:  [6.647144747989269]
<Residue ASP het=  resseq=64 icode= >
[-21.     47.883  23.974]
1
The distances are:  [6.647144747989269, 5.012880552608857]
<Residue ASP het=  resseq=64 icode= >

<Residue GLU het=  resseq=140 icode= >
[11.9   25.077  7.693]
2
The distances are:  [36.916591544402586, 41.023385503249166, 41.03914721822977]
<Residue GLU het=  resseq=140 icode= >
[ 5.337 23.789  9.351]
0
The distances are:  [14.317992058267011]
<Residue GLU het=  resseq=140 icode= >
[11.416 23.758  9.341]
1
The distances are:  [14.317992058267011, 11.128662820433508]
<Residue GLU het=  resseq=140 icode= >
[11.9   25.077  7.693]
2
The distances are:  [14.317992058267011, 11.128662820433508, 13.014388296587745]
<Residue GLU het=  resseq=140 icode= >
[ 5.337 23.789  9.351]
0
The distances are:  [8.507258697929526]
<Residue GLU het=  resseq=140 icode= >
[11.416 23.758  9.341]
1
The distances are:  [8.507258697929526, 2.5721739103216943]
<Residue GLU het=  resseq=140 icode= >
[11.9   25.077  7.693]
2
The distances are:  [8.507258697929526, 2.5721739103216943, 2.3217796724390887]
<Residue GLU het=  resseq=140 icode= >
[<Atom O>, <Atom OD1>]
[11.619 23.618  3.115]
0
The distances are:  [4

[<Atom O>, <Atom OE1>, <Atom OE2>]
[ 8.807 15.229  2.003]
0
The distances are:  [53.17155406769564]
<Residue GLU het=  resseq=123 icode= >
[ 6.719 11.946 -1.511]
1
The distances are:  [53.17155406769564, 56.73262196171518]
<Residue GLU het=  resseq=123 icode= >
[ 8.515 10.817 -0.98 ]
2
The distances are:  [53.17155406769564, 56.73262196171518, 57.92177498445303]
<Residue GLU het=  resseq=123 icode= >
[ 8.807 15.229  2.003]
0
The distances are:  [47.78301876543891]
<Residue GLU het=  resseq=123 icode= >
[ 6.719 11.946 -1.511]
1
The distances are:  [47.78301876543891, 50.61082140561452]
<Residue GLU het=  resseq=123 icode= >
[ 8.515 10.817 -0.98 ]
2
The distances are:  [47.78301876543891, 50.61082140561452, 52.12312513929463]
<Residue GLU het=  resseq=123 icode= >
[ 8.807 15.229  2.003]
0
The distances are:  [17.302679153256445]
<Residue GLU het=  resseq=123 icode= >
[ 6.719 11.946 -1.511]
1
The distances are:  [17.302679153256445, 21.938033380146518]
<Residue GLU het=  resseq=123 icode=

...

[-25.933  33.803  38.324]
1
The distances are:  [21.242129345947607, 24.081058508846066]
<Residue GLU het=  resseq=6 icode= >
[-26.589  31.865  37.662]
2
The distances are:  [21.242129345947607, 24.081058508846066, 25.83388911094594]
<Residue GLU het=  resseq=6 icode= >
[-21.498  34.011  35.421]
0
The distances are:  [22.054710484287725]
<Residue GLU het=  resseq=6 icode= >
[-25.933  33.803  38.324]
1
The distances are:  [22.054710484287725, 25.27135412103801]
<Residue GLU het=  resseq=6 icode= >
[-26.589  31.865  37.662]
2
The distances are:  [22.054710484287725, 25.27135412103801, 26.34768932310231]
<Residue GLU het=  resseq=6 icode= >
[-21.498  34.011  35.421]
0
The distances are:  [43.65141716772233]
<Residue GLU het=  resseq=6 icode= >
[-25.933  33.803  38.324]
1
The distances are:  [43.65141716772233, 48.50017044197388]
<Residue GLU het=  resseq=6 icode= >
[-26.589  31.865  37.662]
2
The distances are:  [43.65141716772233, 48.50017044197388, 48.179854282000775]
<Residue GLU het= 

[-5.731 51.227 30.32 ]
0
The distances are:  [40.139899097810954]
<Residue GLU het=  resseq=31 icode= >
[-11.595  53.888  32.345]
1
The distances are:  [40.139899097810954, 46.04512724974763]
<Residue GLU het=  resseq=31 icode= >
[-11.533  52.987  30.447]
2
The distances are:  [40.139899097810954, 46.04512724974763, 44.45513340947487]
<Residue GLU het=  resseq=31 icode= >
[<Atom O>, <Atom OD1>, <Atom OD2>]
[-12.567  50.29   35.225]
0
The distances are:  [4.9394829883365]
<Residue ASP het=  resseq=20 icode= >
[-15.827  50.153  33.198]
1
The distances are:  [4.9394829883365, 4.231847307411188]
<Residue ASP het=  resseq=20 icode= >
[-14.153  51.558  32.903]
2
The distances are:  [4.9394829883365, 4.231847307411188, 2.3349677405932607]
<Residue ASP het=  resseq=20 icode= >
[-12.567  50.29   35.225]
0
The distances are:  [15.288975254536684]
<Residue ASP het=  resseq=20 icode= >
[-15.827  50.153  33.198]
1
The distances are:  [15.288975254536684, 12.495274197201443]
<Residue ASP het=  resse

The distances are:  [15.993808395696796, 22.466261515438006, 22.248455944020854]
<Residue GLU het=  resseq=84 icode= >
[-0.517 20.391 20.553]
0
The distances are:  [19.1252676853675]
<Residue GLU het=  resseq=84 icode= >
[-6.732 22.092 21.38 ]
1
The distances are:  [19.1252676853675, 24.372925190714685]
<Residue GLU het=  resseq=84 icode= >
[-6.925 20.026 20.673]
2
The distances are:  [19.1252676853675, 24.372925190714685, 24.40798249270602]
<Residue GLU het=  resseq=84 icode= >
[<Atom O>, <Atom OE1>]
[-17.2    34.553  32.513]
0
The distances are:  [19.16028099073825]
<Residue GLN het=  resseq=8 icode= >
[-18.786  29.066  30.177]
1
The distances are:  [19.16028099073825, 24.86085321128821]
<Residue GLN het=  resseq=8 icode= >
[-17.2    34.553  32.513]
0
The distances are:  [19.48398574888263]
<Residue GLN het=  resseq=8 icode= >
[-18.786  29.066  30.177]
1
The distances are:  [19.48398574888263, 23.077940660040223]
<Residue GLN het=  resseq=8 icode= >
[-17.2    34.553  32.513]
0
The di

[-14.153  51.558  32.903]
2
The distances are:  [45.97856018646508, 46.737412767913085, 46.360063490811484]
<Residue ASP het=  resseq=20 icode= >
[<Atom O>, <Atom OG1>]
[-14.199  51.753  29.724]
0
The distances are:  [2.4547302023484066]
<Residue THR het=  resseq=26 icode= >
[-15.954  53.491  28.082]
1
The distances are:  [2.4547302023484066, 4.131089262907538]
<Residue THR het=  resseq=26 icode= >
[-14.199  51.753  29.724]
0
The distances are:  [9.699703431494141]
<Residue THR het=  resseq=26 icode= >
[-15.954  53.491  28.082]
1
The distances are:  [9.699703431494141, 8.166432474982006]
<Residue THR het=  resseq=26 icode= >
[-14.199  51.753  29.724]
0
The distances are:  [46.3898410287994]
<Residue THR het=  resseq=26 icode= >
[-15.954  53.491  28.082]
1
The distances are:  [46.3898410287994, 48.40560948851825]
<Residue THR het=  resseq=26 icode= >
[-14.199  51.753  29.724]
0
The distances are:  [44.90683527062919]
<Residue THR het=  resseq=26 icode= >
[-15.954  53.491  28.082]
1
The 

[11.9   25.077  7.693]
2
The distances are:  [36.916591544402586, 41.023385503249166, 41.03914721822977]
<Residue GLU het=  resseq=140 icode= >
[ 5.337 23.789  9.351]
0
The distances are:  [14.317992058267011]
<Residue GLU het=  resseq=140 icode= >
[11.416 23.758  9.341]
1
The distances are:  [14.317992058267011, 11.128662820433508]
<Residue GLU het=  resseq=140 icode= >
[11.9   25.077  7.693]
2
The distances are:  [14.317992058267011, 11.128662820433508, 13.014388296587745]
<Residue GLU het=  resseq=140 icode= >
[ 5.337 23.789  9.351]
0
The distances are:  [8.507258697929526]
<Residue GLU het=  resseq=140 icode= >
[11.416 23.758  9.341]
1
The distances are:  [8.507258697929526, 2.5721739103216943]
<Residue GLU het=  resseq=140 icode= >
[11.9   25.077  7.693]
2
The distances are:  [8.507258697929526, 2.5721739103216943, 2.3217796724390887]
<Residue GLU het=  resseq=140 icode= >
[<Atom O>, <Atom OD1>]
[16.582 23.13  19.267]
0
The distances are:  [44.455191633160496]
<Residue ASN het=  r

In [31]:
print('The length is: ', len(positionDistFilterans))
positionDistFilterans

The length is:  208


[[<Atom OE1>],
 [<Atom OE1>],
 [<Atom OE1>],
 [<Atom OE1>, <Atom OE2>],
 [],
 [],
 [<Atom OE1>, <Atom OE2>],
 [],
 [],
 [<Atom OE1>, <Atom OE2>],
 [<Atom OE1>, <Atom OE2>],
 [],
 [],
 [],
 [<Atom OE1>, <Atom OE2>],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [<Atom OE2>, <Atom OD1>],
 [<Atom OE2>, <Atom OD1>],
 [<Atom OD1>],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [<Atom OE1>, <Atom OE2>],
 [<Atom OE1>, <Atom OE2>],
 [],
 [<Atom OD2>, <Atom OD1>],
 [<Atom OD2>, <Atom OE1>, <Atom OE2>],
 [<Atom OD2>],
 [<Atom OD2>, <Atom OD1>, <Atom OE1>, <Atom OE2>],
 [],
 [],
 [],
 [],
 [],
 [<Atom OE1>, <Atom OD2>],
 [<Atom OE1>, <Atom OD2>, <Atom OD2>],
 [<Atom OE1>, <Atom OD1>, <Atom OD2>, <Atom OD2>],
 [<Atom OE1>, <Atom OD2>],
 [<Atom OE1>],
 [<Atom OE1>, <Atom OD2>],
 [<Atom OE1>],
 [<Atom OE1>],
 [<Atom OE1>],
 [<Atom OE1>],
 [<Atom O>],
 [<Atom O>],
 [<Atom O>],
 [<Atom O>],
 [<Atom O>, <Atom OD2>, <Atom OD1>, <

In [32]:
# Center of mass coordinates
centreCa2ans = centreCa2(positionDistFilterans)
centreCa2ans

[[-12.427000045776369, 52.81099955240885, 31.898333867390946],
 [-12.82925009727478, 53.31049919128417, 32.272750377655036],
 [14.567666371663412, 17.270000139872234, 18.55833371480306],
 [14.945999622344969, 17.91124987602234, 18.484500408172604],
 [13.645999908447266, 23.251499652862545, 7.942999839782714],
 [14.228000005086262, 22.64299964904785, 8.026333173116047],
 [-17.601000150044758, 49.979166666666664, 20.53316656748454],
 [-16.831000010172524, 51.157666524251304, 20.677333196004234],
 [-17.954333623250324, 51.03733317057292, 19.672332763671875],
 [-17.865250110626224, 49.138500213623054, 20.09049987792969],
 [-17.350666681925457, 48.73100026448568, 20.458333333333336],
 [-13.103200149536132, 52.998999023437506, 31.763000488281246],
 [-12.87000012397766, 52.546499252319336, 31.354750633239746],
 [-12.442333539326984, 52.875999450683594, 30.838667551676433],
 [-12.427000045776369, 52.81099955240885, 31.898333867390953],
 [15.24033292134603, 18.572333017985027, 19.03300031026204

In [33]:
def writeresultAtoms(cacmcoord):
    ans = []
    for coord in cacmcoord:
        print(coord)
        cacoord = "HETATM      CA    CA" + "           " + str(coord[0])[:-12] +"  " + str(coord[1])[:-12] + "  " + str(coord[2])[:-12]
        print(cacoord)
        ans.append(cacoord)
    return ans

In [34]:
filenamearg = './inputdata/'+ PDBID + '.pdb'
metal ='CA'
resultAtoms = writeresultAtoms(centreCa2ans)
runPrediction(filenamearg, metal, PDBID, resultAtoms)

[-12.427000045776369, 52.81099955240885, 31.898333867390946]
HETATM      CA    CA           -12.427  52.81  31.898
[-12.82925009727478, 53.31049919128417, 32.272750377655036]
HETATM      CA    CA           -12.82  53.31  32.272
[14.567666371663412, 17.270000139872234, 18.55833371480306]
HETATM      CA    CA           14.567  17.270  18.55
[14.945999622344969, 17.91124987602234, 18.484500408172604]
HETATM      CA    CA           14.945  17.91  18.484
[13.645999908447266, 23.251499652862545, 7.942999839782714]
HETATM      CA    CA           13.645  23.251  7.942
[14.228000005086262, 22.64299964904785, 8.026333173116047]
HETATM      CA    CA           14.228  22.64  8.026
[-17.601000150044758, 49.979166666666664, 20.53316656748454]
HETATM      CA    CA           -17.601  49.979  20.53
[-16.831000010172524, 51.157666524251304, 20.677333196004234]
HETATM      CA    CA           -16.831  51.157  20.677
[-17.954333623250324, 51.03733317057292, 19.672332763671875]
HETATM      CA    CA         

In [35]:
# def envclean():
#     p = subprocess.Popen('rm -r ./inputdata; rm -r ./outputdata; rm -r ./predictionResults; "Del all needed folders";', shell=True, stdout=subprocess.PIPE)
#     p.wait()
#   # print(str(p.communicate()[0], 'utf-8'))

# envclean()